In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2


def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))  

In [ ]:
import os
import sys

inference_ipynb_path='/root/code/SimpleAICV_pytorch_training_examples/14.video_interactive_segmentation_training/sam2_predict_example'
BASE_DIR = os.path.dirname(os.path.dirname(inference_ipynb_path))
sys.path.append(BASE_DIR)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import torch
import torch.nn as nn
import torch.nn.functional as F

from SimpleAICV.video_interactive_segmentation.models.segment_anything2.sam2image import hiera_l_sam2image
from SimpleAICV.video_interactive_segmentation.common import load_state_dict


sam_checkpoint = '/root/autodl-tmp/pretrained_models/sam2.1_convert_from_pytorch_official_weights/sam2.1_hiera_large_convert_from_pytorch_official_weight.pth'

sam_model = hiera_l_sam2image()
sam_model = sam_model.cuda()
sam_model = sam_model.eval()

load_state_dict(sam_checkpoint,sam_model)

In [ ]:
test_image_path='/root/code/SimpleAICV_pytorch_training_examples/14.video_interactive_segmentation_training/sam2_predict_example/test_images/dogs.jpg'
origin_image = cv2.imdecode(np.fromfile(test_image_path, dtype=np.uint8),
                        cv2.IMREAD_COLOR)
origin_image = cv2.cvtColor(origin_image, cv2.COLOR_BGR2RGB)
print(origin_image.shape,origin_image.dtype)

plt.figure(figsize=(10,10))
plt.imshow(origin_image)
plt.axis('on')
plt.show()

In [ ]:
origin_h, origin_w = origin_image.shape[0], origin_image.shape[1]
h_factor = 1024 / origin_h
w_factor = 1024 / origin_w
resized_image = cv2.resize(origin_image, (1024, 1024))
print(resized_image.shape,resized_image.dtype,h_factor,w_factor)

plt.figure(figsize=(10, 10))
plt.imshow(resized_image)
plt.axis('on')
plt.show()

mean = [123.675, 116.28, 103.53]
std = [58.395, 57.12, 57.375]
norm_image = (resized_image - mean) / std
print(norm_image.shape,np.max(norm_image),np.min(norm_image))

padded_img = np.zeros((1024, 1024, 3),
                        dtype=np.float32)
padded_img[:, :, :] = norm_image
print(padded_img.shape,np.max(padded_img),np.min(padded_img))

padded_img = torch.tensor(padded_img).float().cuda().permute(2, 0, 1).unsqueeze(0)
print(padded_img.shape,torch.max(padded_img),torch.min(padded_img))

In [ ]:
grid_size = 32
positive_grid_points = []
positive_grid_labels = []
for i in range(grid_size):
    per_point_x = 0.5 + i / grid_size * origin_w
    for j in range(grid_size):
        per_point_y = 0.5 + j / grid_size * origin_h
        positive_grid_points.append([per_point_x, per_point_y])
        positive_grid_labels.append([1])

positive_grid_points = np.array(positive_grid_points)
positive_grid_points[:,0] = positive_grid_points[:,0]*w_factor
positive_grid_points[:,1] = positive_grid_points[:,1]*h_factor
positive_grid_labels = np.array(positive_grid_labels)
print(grid_size,origin_h,origin_w,positive_grid_points.shape,positive_grid_labels.shape)

prompt_points = np.concatenate([positive_grid_points,positive_grid_labels],axis=1)
print(prompt_points.shape)

plt.figure(figsize=(10,10))
plt.imshow(resized_image)
show_points(positive_grid_points, np.squeeze(positive_grid_labels,axis=1), plt.gca())
plt.axis('on')
plt.show()

In [ ]:
per_images = padded_img.clone()

with torch.no_grad():
    per_image_embedding = sam_model.forward_image_encoder(per_images)
print(per_image_embedding[0].shape, per_image_embedding[1].shape, per_image_embedding[2].shape)

batch_mask_preds = []
batch_iou_preds = []
for per_point in prompt_points:
    per_point = np.expand_dims(per_point, axis=0)
    per_prompt_point = torch.tensor(np.expand_dims(per_point,
                                                   axis=0)).float().cuda()
    per_prompt = {
        'prompt_point': per_prompt_point,
        'prompt_box': None,
        'prompt_mask': None
    }
    mask_out_idxs = [0]
    with torch.no_grad():
        mask_preds, iou_preds = sam_model.forward_prompt_encoder_mask_decoder(
            per_image_embedding, per_prompt, mask_out_idxs=mask_out_idxs)
        mask_preds, iou_preds = mask_preds[0], iou_preds[0]

    batch_mask_preds.append(mask_preds)
    batch_iou_preds.append(iou_preds)
print(len(batch_mask_preds), batch_mask_preds[0].shape, len(batch_iou_preds), batch_iou_preds[0].shape)

batch_mask_preds = torch.cat(batch_mask_preds, dim=0)
batch_iou_preds = torch.cat(batch_iou_preds, dim=0)
batch_mask_preds = batch_mask_preds.cpu().numpy()
batch_iou_preds = batch_iou_preds.cpu().numpy()

# [mask_num,mask_h,mask_w],[mask_num]
print(batch_mask_preds.shape, batch_iou_preds.shape, batch_iou_preds[0])

In [ ]:
def calculate_masks_stability_score(masks, mask_threshold, threshold_offset):
    """
    Computes the stability score for a batch of masks. The stability
    score is the IoU between the binary masks obtained by thresholding
    the predicted mask logits at high and low values.
    """
    intersections = np.sum(np.sum((masks >
                                   (mask_threshold + threshold_offset)),
                                  axis=-1),
                           axis=-1)
    unions = np.sum(np.sum((masks > (mask_threshold - threshold_offset)),
                           axis=-1),
                    axis=-1)

    return intersections / unions


def filter_small_area_masks(masks, ious, filter_area_ratio=0.0001):
    per_mask_h, per_mask_w = masks.shape[1], masks.shape[2]
    masks_area = np.sum(masks, axis=(1, 2))

    keep_masks = []
    keep_ious = []
    for per_mask_area, per_mask, per_iou in zip(masks_area, masks, ious):
        per_mask_area_ratio = per_mask_area / float(per_mask_h * per_mask_w)

        if per_mask_area_ratio > filter_area_ratio:
            keep_masks.append(per_mask)
            keep_ious.append(per_iou)

    keep_masks = np.array(keep_masks)
    keep_ious = np.array(keep_ious)

    return keep_masks, keep_ious


def fill_masks_hole(masks, fill_area_threshold=100):
    filled_masks = []
    for per_mask in masks:
        per_mask = per_mask.astype(np.uint8)
        # 取反mask,因为我们要填充的是前景中的空洞
        mask_invert = 1 - per_mask

        # 寻找连通区域
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(
            mask_invert, 8, cv2.CV_32S)

        # Row 0 is background label
        sizes = stats[:, -1][1:]
        small_regions = [
            i + 1 for i, s in enumerate(sizes) if s < fill_area_threshold
        ]
        if len(small_regions) == 0:
            filled_masks.append(per_mask)
        else:
            fill_labels = [0] + small_regions
            per_mask = np.isin(labels, fill_labels).astype(np.uint8)
            filled_masks.append(per_mask)

    filled_masks = np.array(filled_masks)

    return filled_masks


def calculate_mask_nms(sorted_masks, sorted_scores, nms_threshold=0.5):
    '''
    sorted_masks:[mask_nums,mask_h,mask_w]
    sorted_scores:[mask_nums],mask predict scores
    '''
    sorted_masks_area = np.sum(sorted_masks, axis=(1, 2))

    indexes = np.array([i for i in range(sorted_scores.shape[0])],
                       dtype=np.int32)

    keep = []
    while indexes.shape[0] > 0:
        keep_idx = indexes[0]
        keep.append(keep_idx)
        indexes = indexes[1:]
        if len(indexes) == 0:
            break

        keep_mask_area = sorted_masks_area[keep_idx]

        overlap_mask_area = np.sum((sorted_masks[keep_idx]
                                    & sorted_masks[indexes]),
                                   axis=(1, 2))
        union_mask_area = keep_mask_area + sorted_masks_area[
            indexes] - overlap_mask_area
        union_mask_area = np.maximum(union_mask_area, 1e-4)
        ious = overlap_mask_area / union_mask_area

        candidate_indexes = np.where(ious < nms_threshold)[0]
        indexes = indexes[candidate_indexes]

    keep = np.array(keep)

    return keep

In [ ]:
iou_pred_filter_threshold = 0.9
pred_masks = batch_mask_preds[batch_iou_preds > iou_pred_filter_threshold]
pred_ious = batch_iou_preds[batch_iou_preds > iou_pred_filter_threshold]
print('1111',pred_masks.shape,pred_ious.shape,pred_ious[0])

mask_stability_score_filter_threshold=0.9
pred_mask_stability_scores = calculate_masks_stability_score(pred_masks, 0.0, 1.0)
pred_masks = pred_masks[pred_mask_stability_scores>mask_stability_score_filter_threshold]
pred_ious = pred_ious[pred_mask_stability_scores>mask_stability_score_filter_threshold]
print('2222',pred_masks.shape,pred_ious.shape,pred_ious[0])

pred_masks= (pred_masks > 0).astype(np.uint8)
print('3333',pred_masks.shape,pred_ious.shape,pred_ious[0],np.unique(pred_masks))

fill_area_threshold=100
pred_masks=fill_masks_hole(pred_masks,fill_area_threshold=fill_area_threshold).astype(np.uint8)
print('4444',pred_masks.shape,pred_ious.shape,pred_ious[0],np.unique(pred_masks))

filter_area_ratio=0.0001
pred_masks,pred_ious=filter_small_area_masks(pred_masks, pred_ious, filter_area_ratio=filter_area_ratio)
print('5555',pred_masks.shape,pred_ious.shape,pred_ious[0],np.unique(pred_masks))

In [ ]:
# descending sort
iou_sorted_indexes = np.argsort(-pred_ious)
pred_masks = pred_masks[iou_sorted_indexes]
pred_ious = pred_ious[iou_sorted_indexes]
print('6666',pred_masks.shape,pred_ious.shape,pred_ious[0])

nms_threshold=0.3
keep_mask_indexes=calculate_mask_nms(pred_masks, pred_ious, nms_threshold=nms_threshold)
pred_masks = pred_masks[keep_mask_indexes]
pred_ious = pred_ious[keep_mask_indexes]
print('7777',keep_mask_indexes.shape,pred_masks.shape,pred_ious.shape)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(resized_image)
for i, per_mask in enumerate(pred_masks):
    show_mask(per_mask, plt.gca(), random_color=True)
plt.axis('off')
plt.show()  